We use a product review dataset

In [ ]:
!wget https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/PROD.csv

--2022-05-12 15:29:35--  https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/PROD.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 527639 (515K) [text/plain]
Saving to: ‘PROD.csv’

PROD.csv            100%[===================>] 515.27K  --.-KB/s    in 0.02s   

2022-05-12 15:29:35 (21.7 MB/s) - ‘PROD.csv’ saved [527639/527639]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
import tensorflow as tf
import pandas as pd
import re
import numpy as np
import csv 
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle


## Read the Dataset

preprocess a review by removing special characters and long spaces

In [ ]:
def process_review(review):
  out = re.sub(r"[^\w\s]", '', review)
  out = re.sub(r"[a-zA-Z]", '', out)
  out = re.sub(r"\n", '', out)
  out = re.sub(r"\s+", ' ', out)
  return out.strip()

In [ ]:
with open('/content/drive/MyDrive/new_ar_reviews_100k.csv', 'r') as csv_file:
  reviews = []
  labels  = []
  all_text = ""
  count = 0
  pos_count = 0
  
  #read the data
  lines = csv.reader(csv_file, delimiter = ",")
  for i, line in enumerate(lines):
    
    #ignore the first line
    if i == 0: continue
    
    #preprocess the data
    review = process_review(line[1])
    label  = int(line[0])
    
    
    #only allow postiive and negative reviews, 
    #also make them the same length
    
    if label == 1:
      pos_count +=1
    elif label == 0:
      continue
    else:
      label += 1
      
    if label == 1 and pos_count > 99998:
      continue
    
    if review == "":
      continue
    reviews.append(review)
    all_text += review +' \n '
    labels.append(label)
    
#shuffle the data
reviews, labels = shuffle(reviews, labels)
print(len(reviews))

66663


Look at the data

In [ ]:
for i in range(0, 10):
  print(reviews[i], labels[i])

بسيط صغير مسلي مفيد مختصر جدا 0
أنهيت الكتاب خلال يومين كانت تجربة رائعة جدا لي مع الكتب المسموعة جدا رائعة التجربة الكتاب كان جيد كثيرا نالت من فكرة أننا نعيش طويلاا الكثير هل ذلك يعتبر علميا المخ و سجله التاريخي الطويل هل كنت سيدة هندية أو تاجر فارسي قديم أنا لا أعلم هل سأكون بعد ذلك إمبراطورة ما مستقبلا جميلة جدا الرواية لمن يريد سماع الكتاب و كتب أخرى 1
من أروع ما قرأت أقول ذلك على الكتاب الذى أقرؤه و أتعلم منه شىء و هذا الكتاب تعلمت منه الكثير ما اكثر مداخل الحب فى العلاقة بالله و من خلال الحوارات الدائرة بين شخصيات الرواية عرفت معنى قراءة القرأن بالروح رحلات عبر الزمن الذى بيد الله وحده فقد خلق الله اماكن كالشمس و القمر لتنتج بحوله و قوته الزمن اشارات ايمانية كثيرة تجعل الطاقة الايمانية أقوى من أن يتصور أحد ثروت الخرباوى تفوقت على نفسك كثيرا فى هذه الرواية الرائعة و مع الاستمرارية اعتقد فى رأى الشخصى المتواضع ستصبح أديبا و الجدير بالذكر انك بطريقة ما ما زلت متأثرا بسر المعبد و هذا ليس عيبا بالتأكيد لأنك تفجر قضية فى منتهى الخطورة فى زمكان و هى قضية تجار الدين و الفهم الخاطىء للدي

In [ ]:
#save the dataset
with open('newdataset.txt', 'w') as f:
  for i in range(len(reviews)):
    f.write(f"{reviews[i]}, {labels[i]} '\n'")

## Create Sequences
Create sequences by using the most repeated 500 words

In [ ]:
tknzr = Tokenizer(lower=True, split=" ")
tknzr.fit_on_texts(reviews)

#making sequences:
X = tknzr.texts_to_sequences(reviews)
X = pad_sequences(X, padding='post', value=0)

## Create Numpy Arrays

In [ ]:
X = np.array(X)
y = np.array(labels)

print(X.shape)

(66663, 1575)


In [ ]:
from sklearn.model_selection import train_test_split
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

## Create the model

In [ ]:
model = Sequential()
model.add(Embedding(len(tknzr.word_index), 32))
model.add(Bidirectional(GRU(units = 32)))
model.add(Dense(32, activation = 'tanh'))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy',keras.metrics.Precision(), keras.metrics.Recall()])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 32)          8305120   
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               12672     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 8,319,905
Trainable params: 8,319,905
Non-trainable params: 0
____________________________________________

## Train the model

In [ ]:
model.fit(X_train, y_train, validation_split = 0.1, epochs = 7, batch_size= 128, shuffle = True)

Epoch 1/7
329/329 [==============================] - 37s 101ms/step - loss: 0.5374 - accuracy: 0.7051 - precision_2: 0.7058 - recall_8: 0.7061 - val_loss: 0.5006 - val_accuracy: 0.7358 - val_precision_2: 0.7738 - val_recall_8: 0.6526
Epoch 2/7
329/329 [==============================] - 32s 99ms/step - loss: 0.3324 - accuracy: 0.8553 - precision_2: 0.8567 - recall_8: 0.8541 - val_loss: 0.5915 - val_accuracy: 0.7281 - val_precision_2: 0.7586 - val_recall_8: 0.6543
Epoch 3/7
329/329 [==============================] - 32s 99ms/step - loss: 0.1444 - accuracy: 0.9445 - precision_2: 0.9441 - recall_8: 0.9452 - val_loss: 0.8662 - val_accuracy: 0.7114 - val_precision_2: 0.7282 - val_recall_8: 0.6574
Epoch 4/7
329/329 [==============================] - 33s 99ms/step - loss: 0.0751 - accuracy: 0.9741 - precision_2: 0.9733 - recall_8: 0.9751 - val_loss: 1.0954 - val_accuracy: 0.7011 - val_precision_2: 0.6998 - val_recall_8: 0.6849
Epoch 5/7
329/329 [==============================] - 33s 100ms/step

## Tests

In [ ]:
class_names = ['Negative' , 'Positive']
def classify(sentence):
  sentence = process_review(sentence)
  sequence = [tknzr.word_index[word] for word in sentence.split(' ')]
  sequence = pad_sequences([sequence], maxlen = X.shape[1], padding='post', value=0)
  #print(sequence.dtype)
  #print(sequence)
  pred = model.predict(sequence)[0][0]
  print(class_names[np.round(pred).astype('int')], pred)
  

In [ ]:
classify("جميل")

Positive 0.6028088


In [ ]:
classify("السلعة كانت جيدة")

Negative 0.032516178


In [ ]:
classify("سيء")

Negative 0.0011310811


In [ ]:
classify("لا بأس بها")

Negative 0.028060263


In [ ]:
classify("رائع")

Positive 0.9977114


In [ ]:
import csv
def create_csv(file, dict):
    with open(file, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for key in dict.keys():
            writer.writerow([key,dict[key]])

In [ ]:
create_csv("word2index.csv", tknzr.word_index)

In [ ]:
model.save("Rclass1.h5")